Loading useful packages

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.model_selection import train_test_split
import networkx as nx
import collections
import math
import matplotlib.pyplot as plt
import glob
from sklearn.preprocessing import StandardScaler

Loading data sets and combining them

In [2]:
g = glob.glob('F:\\SWaT Dataset\\*.csv')

In [3]:
dataframes = []

for filename in g[490:500]:
    print("loading file: {}".format(filename))
    dataframes.append(pd.read_csv(filename,error_bad_lines = False, sep=","))

loading file: F:\SWaT Dataset\2015-12-29_190411_104.log.part01_sorted.csv
loading file: F:\SWaT Dataset\2015-12-29_190411_104.log.part02_sorted.csv
loading file: F:\SWaT Dataset\2015-12-29_190411_104.log.part03_sorted.csv
loading file: F:\SWaT Dataset\2015-12-29_190411_104.log.part04_sorted.csv
loading file: F:\SWaT Dataset\2015-12-29_190411_104.log.part05_sorted.csv
loading file: F:\SWaT Dataset\2015-12-29_190411_104.log.part06_sorted.csv
loading file: F:\SWaT Dataset\2015-12-29_190411_104.log.part07_sorted.csv
loading file: F:\SWaT Dataset\2015-12-29_190411_104.log.part08_sorted.csv
loading file: F:\SWaT Dataset\2015-12-29_190411_104.log.part09_sorted.csv
loading file: F:\SWaT Dataset\2015-12-29_190411_104.log.part10_sorted.csv


In [4]:
df = pd.concat(dataframes,ignore_index=True)
df.head()

,num,date,time,orig,type,i/f_name,i/f_dir,src,dst,proto,appi_name,proxy_src_ip,Modbus_Function_Code,Modbus_Function_Description,Modbus_Transaction_ID,SCADA_Tag,Modbus_Value,service,s_port,Tag
0,1,29Dec2015,13:39:41,192.168.1.48,log,eth1,outbound,192.168.1.10,192.168.1.20,tcp,CIP_read_tag_service,192.168.1.10,76.0,Read Tag Service,62639.0,HMI_FIT201,Number of Elements: 1,44818.0,54592.0,0
1,2,29Dec2015,13:39:41,192.168.1.48,log,eth1,outbound,192.168.1.30,192.168.1.40,tcp,CIP_read_tag_service,192.168.1.30,76.0,Read Tag Service - Response,7644.0,HMI_LIT401,0x03 0xa7 0x66 0x44; 0x00 0x00 0x00 0x00; 0x00...,44818.0,52544.0,0
2,3,29Dec2015,13:39:41,192.168.1.48,log,eth1,outbound,192.168.1.60,192.168.1.10,tcp,CIP_read_tag_service,192.168.1.60,76.0,Read Tag Service,13765.0,HMI_LIT101,Number of Elements: 1,44818.0,53260.0,0
3,4,29Dec2015,13:39:41,192.168.1.48,log,eth1,outbound,192.168.1.60,192.168.1.20,tcp,CIP_read_tag_service,192.168.1.60,76.0,Read Tag Service,57718.0,HMI_AIT202,Number of Elements: 1,44818.0,53250.0,0
4,5,29Dec2015,13:39:41,192.168.1.48,log,eth1,outbound,192.168.1.60,192.168.1.10,tcp,CIP_read_tag_service,192.168.1.60,76.0,Read Tag Service - Response,13765.0,HMI_LIT101,0x44 0x5e 0x02 0x44; 0x00 0x00 0x00 0x00; 0x00...,44818.0,53260.0,0


Dropping columns which have no effect on the analysis

In [5]:
df = df.drop([ "date", "time",'num'], axis=1)
df = df[df['i/f_dir']=='outbound']
df = df[df['src'].notna()]
df = df[df['dst'].notna()]
df = df[df['orig'].notna()]
df = df[df['proxy_src_ip'].notna()]
df = df[df['Modbus_Function_Code'].notna()]
df = df[df['Modbus_Function_Description'].notna()]
df = df[df['Modbus_Transaction_ID'].notna()]
df = df[df['SCADA_Tag'].notna()]
df = df[df['appi_name'].notna()]
df = df[df['proto'].notna()]
df = df[df['type'].notna()]
df = df[df['Modbus_Value'].notna()]

Loading packages to convert hexadecimal to numeric using our function and a function in a research paper

In [6]:
import binascii, struct

In [7]:
def modbus_conversion(value):
    lst = []
    for i in range(len(value.split(";"))):
        x = value.split(";")[i]
        if " " in x:
            x = x.replace(" ","")
        if '0x' in x:
            x = x.replace("0x","")
            x = struct.unpack('<f',binascii.unhexlify(x))[0]
        lst.append(x)
    return lst[0]
    
            
        
    

In [8]:
df['Modbus_Value'] = list(map(modbus_conversion,df['Modbus_Value']))

In [11]:
df = df.drop(['i/f_dir','i/f_name'],axis=1)

In [12]:
df = df[df['Modbus_Value'] != 'Number of Elements: 1']

Converting IP addresses to numeric values

In [13]:
import socket, struct

def ip2long(ip):
    """
    Convert an IP string to long
    """
    packedIP = socket.inet_aton(ip)
    return struct.unpack("!L", packedIP)[0]


df['src'] = list(map(ip2long,df['src']))
df['dst'] = list(map(ip2long,df['dst']))
df['orig'] = list(map(ip2long,df['orig']))
df['proxy_src_ip'] = list(map(ip2long,df['proxy_src_ip']))

df[['src','dst','orig','proxy_src_ip','Modbus_Value','Modbus_Function_Code','Modbus_Transaction_ID','s_port','service']] = StandardScaler().fit_transform(df[['src','dst','orig','proxy_src_ip','Modbus_Value','Modbus_Function_Code','Modbus_Transaction_ID','s_port','service']])

C:\Users\KISHALAY\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:617: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\KISHALAY\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [48]:
df.head()

,orig,type,src,dst,proto,appi_name,proxy_src_ip,Modbus_Function_Code,Modbus_Function_Description,Modbus_Transaction_ID,SCADA_Tag,Modbus_Value,service,s_port,Tag
0,0.0,log,-1.158609,-0.332403,tcp,CIP_read_tag_service,-1.158609,0.0,Read Tag Service,1.578958,HMI_FIT201,-0.755708,0.0,1.553161,0
1,0.0,log,-0.210294,1.663971,tcp,CIP_read_tag_service,-0.210294,0.0,Read Tag Service - Response,-1.327896,HMI_LIT401,-0.717512,0.0,-1.325169,0
2,0.0,log,1.212179,-1.330589,tcp,CIP_read_tag_service,1.212179,0.0,Read Tag Service,-1.004360,HMI_LIT101,-0.755708,0.0,-0.318878,0
3,0.0,log,1.212179,-0.332403,tcp,CIP_read_tag_service,1.212179,0.0,Read Tag Service,1.318850,HMI_AIT202,-0.755708,0.0,-0.332932,0
4,0.0,log,1.212179,-1.330589,tcp,CIP_read_tag_service,1.212179,0.0,Read Tag Service - Response,-1.004360,HMI_LIT101,-0.087350,0.0,-0.318878,0


Applying random under sampling to majority data type and randomly reshuffling observations to try and eliminate latent time dependency

In [15]:
tag_1 = df[df['Tag'] == 1]
tag_0 = df[df['Tag'] == 0]

print(len(tag_1))
print(len(tag_0))

a = len(tag_1)

sample_1 = tag_1.sample(a)
sample_0 = tag_0.sample(a)

sample_set = pd.concat([sample_0,sample_1], ignore_index = True)
#random_sample = sample_set.sample(546850,replace=False)
sample_set = sample_set.sample(2*a,replace=False)

sample_set['SCADA_Tag'] = pd.factorize(sample_set.SCADA_Tag)[0]
sample_set['Modbus_Function_Description'] = pd.factorize(sample_set.Modbus_Function_Description)[0]
sample_set['appi_name'] = pd.factorize(sample_set.appi_name)[0]
sample_set['proto'] = pd.factorize(sample_set.proto)[0]
sample_set['type'] = pd.factorize(sample_set.type)[0]



110185
2235010


Train Test Split

In [16]:
y = sample_set['Tag']
sample_set = sample_set.drop('Tag',axis=1)
x = sample_set.values
X_train, X_test, y_train, y_test =train_test_split(x,y, test_size = 0.25)

Loading useful packages and writing the neural networks model

In [17]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
from keras.utils import to_categorical
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from scipy import stats
# fix random seed for reproducibility
np.random.seed(7)

Using TensorFlow backend.


In [18]:
model = Sequential()
model.add(Dense(56, input_dim=14, activation='sigmoid'))
model.add(Dense(56, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=10000)
scaledtorscores = model.evaluate(X_train, y_train)
#loss.append(scaledtorscores[0])
#acc.append(scaledtorscores[1]*100)
#print("\n%s: %.2f%%" % (model.metrics_names[1], scaledtorscores[1]*100))

Epoch 1/10
165277/165277 [==============================] - 1s 5us/step - loss: 0.7080 - acc: 0.5032
Epoch 2/10
165277/165277 [==============================] - 0s 1us/step - loss: 0.6910 - acc: 0.5197
Epoch 3/10
165277/165277 [==============================] - 0s 1us/step - loss: 0.6863 - acc: 0.5341
Epoch 4/10
165277/165277 [==============================] - 0s 1us/step - loss: 0.6833 - acc: 0.5606
Epoch 5/10
165277/165277 [==============================] - 0s 1us/step - loss: 0.6808 - acc: 0.5542
Epoch 6/10
165277/165277 [==============================] - 0s 1us/step - loss: 0.6787 - acc: 0.5526
Epoch 7/10
165277/165277 [==============================] - 0s 2us/step - loss: 0.6773 - acc: 0.5585
Epoch 8/10
165277/165277 [==============================] - 0s 1us/step - loss: 0.6764 - acc: 0.5682
Epoch 9/10
165277/165277 [==============================] - 0s 1us/step - loss: 0.6760 - acc: 0.5696
Epoch 10/10
165277/165277 [==============================] - 2s 10us/step


Generating predictions on the basis of the test data

In [19]:
predictions = model.predict(X_test)

lt = [1]*len(y_test)

for i in range(len(predictions)):
    if predictions[i] >= 0.5:
        lt[i] = 1
    else:
        lt[i] = 0
        

matrix = confusion_matrix(y_test,lt)
matrix

array([[ 6074, 21591],
       [ 2344, 25084]], dtype=int64)

Calculating Sensitivity

In [21]:
stat = (matrix[1][1])/(matrix[1][1]+matrix[0][1])
stat

0.5374183181574719